In [19]:
from pandas import DataFrame
from pandas import read_csv
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from matplotlib import pyplot
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.preprocessing import StandardScaler

In [22]:
dataset = read_csv('with_timestamp.csv', header=0, index_col=0)
dataset.drop("@timestamp", axis=1, inplace=True)
dataset.head()

,system.cpu.idle.pct,system.cpu.iowait.pct,system.cpu.softirq.pct,system.cpu.system.pct,system.cpu.total.pct,system.cpu.user.pct,system.diskio_sda.iostat.await,system.diskio_sda.iostat.busy,system.diskio_sda.iostat.queue.avg_size,system.diskio_sda.iostat.read.request.merges_per_sec,...,system.load.1,system.load.15,system.load.5,system.load.norm.1,system.load.norm.15,system.load.norm.5,system.memory.actual.used.pct,system.memory.swap.used.pct,system.memory.used.pct,anomaly
0,3.5818,0.0474,0.0312,0.1058,0.4182,0.2338,0.698,12.601,0.165,0.0,...,0.68,0.97,1.00,0.170,0.243,0.250,0.6904,0.0,0.9620,0
1,3.5080,0.0934,0.0271,0.1054,0.4920,0.2661,0.572,11.104,0.137,0.0,...,0.58,0.96,0.97,0.145,0.240,0.243,0.6903,0.0,0.9626,0
2,3.5337,0.1136,0.0291,0.1025,0.4663,0.2211,0.572,11.104,0.137,0.0,...,0.65,0.96,0.97,0.163,0.240,0.243,0.6904,0.0,0.9628,0
3,3.5742,0.0503,0.0201,0.0916,0.4258,0.2637,0.572,11.104,0.137,0.0,...,0.68,0.95,0.95,0.170,0.238,0.238,0.6903,0.0,0.9635,0
4,3.5454,0.0705,0.0268,0.1051,0.4546,0.2522,0.572,11.104,0.137,0.0,...,0.68,0.95,0.95,0.170,0.238,0.238,0.6905,0.0,0.9638,0


## Normalizing Data

In [23]:
dataset[dataset.columns] = StandardScaler().fit_transform(dataset)

## Shifting the dataset
shifting only one minute

In [24]:
dataset_shifted = dataset.shift(-1)
dataset = dataset.iloc[:-1]
dataset_shifted = dataset_shifted.iloc[:-1]

## Spliting the dataset 
Ratio - 80 : 20

In [25]:
train = dataset.shape[0] * 2 // 10
    
train_dataset = dataset.iloc[:-train]
train_dataset_shifted = dataset_shifted.iloc[:-train]

test_dataset = dataset.iloc[-train:]
test_dataset_shifted = dataset_shifted.iloc[-train:]

In [26]:
train_dataset = train_dataset.values.reshape((train_dataset.shape[0], 1, train_dataset.shape[1]))
print(train_dataset_shifted.shape)

(40000, 40)


## Exhautive search for optimal parameters

In [31]:
def fit_lstm_tuning(activation, recurrent_activation, loss, optimizer):
    model_lstm = Sequential()
    model_lstm.add(LSTM(1, input_shape=(1, dataset.shape[1]), activation = activation, recurrent_activation = recurrent_activation))
    model_lstm.add(Dense(40))
    model_lstm.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])
    return model_lstm

activations = ['softmax', 'elu', 'selu', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
recurrent_activations = ['softmax', 'elu', 'selu', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']

optimizers = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
losses = ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error', 'squared_hinge', 'hinge', 'categorical_hinge', 'logcosh', 'categorical_crossentropy', 'sparse_categorical_crossentropy', 'binary_crossentropy', 'kullback_leibler_divergence', 'poisson', 'cosine_proximity']

X = train_dataset
Y = train_dataset_shifted

## Create model

In [32]:
model = KerasClassifier(build_fn=fit_lstm_tuning)

param_grid = dict(activation = activations, recurrent_activation = recurrent_activations, optimizer = optimizers, loss = losses)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X,Y)

KeyboardInterrupt: 

## Summarize results

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))